In [41]:
import pandas as pd
import datetime
from configparser import ConfigParser
import json, requests, datetime
from pysimplicate import Simplicate

DATE_FORMAT = '%Y-%m-%d'
# Pandas
pd.options.display.float_format = '{:,.1f}'.format
pd.set_option('display.max_columns', 500)
PANDAS_FILE = '../simplicate_cache/hours.pd'
df = pd.read_pickle(PANDAS_FILE)

df = df.query( 'type=="normal"')
df['turnover'] = df.apply(lambda a: (a['hours']+a['corrections']) * (a['tariff'] if a['tariff'] > 0 else a['service_tariff']), axis=1)
df['turnover'] = df.apply(lambda a: a['turnover']/2 if a['project_number'] == 'TOR-3' else a['turnover'], axis=1)
df['week'] = df.apply(lambda a: datetime.datetime.strptime(a['day'],'%Y-%m-%d').isocalendar()[1], axis=1)
df['month'] = df.apply(lambda a: datetime.datetime.strptime(a['day'],'%Y-%m-%d').month, axis=1)
df['corrections_value'] = df.apply(lambda a: (a['corrections']) * (a['tariff'] if a['tariff'] > 0 else a['service_tariff']), axis=1)

# Simplicate
ini = ConfigParser()
ini.read('../credentials.ini')

subdomain = ini['simplicate']['subdomain']
api_key = ini['simplicate']['api_key']
api_secret = ini['simplicate']['api_secret']

sim = Simplicate(subdomain, api_key, api_secret )

In [42]:
df[-3:]

,employee,organization,project_id,project_name,project_number,service,service_id,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,corrections_value,month
24103,Vinz Timmermans,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,Teambuilding,service:4a12cabaa852fa01d8107a68a60f150e,normal,0.0,Internal,False,0.0,0.8,2021-08-25,to_forward,0.0,0.0,34,0.0,8
24104,Vinz Timmermans,Oberon,project:dac6961c7d699864feaad60b7a7437df,Service Team,SERVIC-1,Meetings,service:f0b810be8dd3aaddb48628b99c52fe43,normal,0.0,Internal,False,0.0,0.2,2021-08-25,to_forward,0.0,0.0,34,0.0,8
24105,Vinz Timmermans,Sprout Money BV,project:d546df523896bc3afeaad60b7a7437df,Marktgevoel Agile,MG2021,Marktgevoel - Augustus 2021,service:599b7052835bc71df0d06734914aca76,normal,110.0,Project Management,True,85.0,0.2,2021-08-25,to_forward,0.0,21.2,34,0.0,8


## Turnover

In [43]:
def turnover( project, month=None, from_date=None ):
    query = f'project_number=="{project}"'
    if month: 
        query += f' and month=={month}'
    if from_date:
        query += f' and day>="{from_date}"'
    data = df.query(query)
    return data['turnover'].sum()

turnover( 'BAM-1', from_date='2021-02-01')

180505.41250000003

## Invoices

In [44]:
def invoiced(project, month=None ):
    filter = {'project_number':project}
    if month:
        filter['from_date'] = f'2021-0{month}-01'
        filter['until_date'] = f'2021-0{month+1}-01'
    invoices = sim.invoice( filter )
    tot = 0
    for invoice in invoices:
        #print( invoice.get('invoice_number','????'), invoice['total_excluding_vat'], invoice['status'])
        tot += invoice['total_excluding_vat']
    return tot

def last_invoice_date( project ):
    invoices = sim.invoice( {'project_number':project} )
    invoices = sorted( invoices, key=lambda i: i['date'])
    if invoices:
        return datetime.datetime.strptime( invoices[-1]['date'], '%Y-%m-%d').date()
    
#print( invoiced( 'BAM-1'))
print( last_invoice_date( 'SLIM-16' ))

2020-11-30


## All projects, certain month

In [45]:
def active_projects():
    projects = [{'project': project.get('project_number',''),
                 'spent' : project['budget']['hours'].get('value_spent', 0),
                 'invoiced' : project['budget']['total']['value_invoiced']
                }
                for project in sim.project( {'active':True} )]
    return projects
active_projects()[:3]

[{'project': 'BAM-1', 'spent': 207415.6375, 'invoiced': 187204.48},
 {'project': 'SPIR-3', 'spent': 0, 'invoiced': 1044},
 {'project': 'WEI-001', 'spent': 0, 'invoiced': 4926.72}]

## Onderhanden werk

In [46]:
pd.set_option('display.max_row', 150)

# project['budget']['total']['value_invoiced']
def corrections(p):
    project = p['project']
    h = df.query( f'project_number=="{project}"' )
    print( 'corrections', project, h['corrections_value'].sum())
    return h['corrections_value'].sum()
    
def onderhanden():
    return pd.DataFrame( [{'project':project['project'], 
                           'spent':project['spent'],
                           'corr': corrections( project),
                           'inv': project['invoiced'],
                           'OH':project['spent'] + corrections( project) - project['invoiced']} 
                          for project in active_projects()] ).sort_values( by=['OH'])
oh = onderhanden()
oh.drop( oh[(oh.project=='TOR-3')].index, inplace=True)
oh

corrections BAM-1 -3662.9625
corrections BAM-1 -3662.9625
corrections SPIR-3 0.0
corrections SPIR-3 0.0
corrections WEI-001 0.0
corrections WEI-001 0.0
corrections VERH-1 -2855.625
corrections VERH-1 -2855.625
corrections OBE-1 0.0
corrections OBE-1 0.0
corrections THIE-26 0.0
corrections THIE-26 0.0
corrections SM2021 85.0
corrections SM2021 85.0
corrections MG2021 -6481.25
corrections MG2021 -6481.25
corrections NUNOT-1 -3585.0
corrections NUNOT-1 -3585.0
corrections SERVIC-1 0.0
corrections SERVIC-1 0.0
corrections VJ2021 -2890.0
corrections VJ2021 -2890.0
corrections BITF-1 0.0
corrections BITF-1 0.0
corrections IDFA-4 -2280.0
corrections IDFA-4 -2280.0
corrections EASY-2 -5652.5
corrections EASY-2 -5652.5
corrections COL-1 -62.5
corrections COL-1 -62.5
corrections AMS-1 -14905.0
corrections AMS-1 -14905.0
corrections ADY-1 0.0
corrections ADY-1 0.0
corrections BRA-2 0.0
corrections BRA-2 0.0
corrections MLA-6 0.0
corrections MLA-6 0.0
corrections TOR-3 -14064.375
corrections TOR-3

,project,spent,corr,inv,OH
6,SM2021,"110,563.8",85.0,"142,290.0","-31,641.2"
83,TRAV-5,0.0,0.0,"17,395.0","-17,395.0"
41,TEX-2,"2,617.5",-585.0,"18,808.5","-16,776.0"
59,HAVA-2,"107,457.5","-26,902.5","90,227.5","-9,672.5"
42,ESCH-1,"15,050.0","-8,030.0","16,640.0","-9,620.0"
50,SLIM-30,"31,705.0",-85.0,"40,800.0","-9,180.0"
11,BITF-1,"11,730.0",0.0,"20,537.8","-8,807.8"
67,TRAV-3,0.0,0.0,"7,927.5","-7,927.5"
63,TRAV-7,0.0,0.0,"7,608.0","-7,608.0"
60,VOLK-1,"18,488.2","-4,974.8","21,047.2","-7,533.8"


In [47]:
oh['OH'].sum()

-3153.343337996688

# Correcties

In [48]:
df[-3:]

,employee,organization,project_id,project_name,project_number,service,service_id,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,corrections_value,month
24103,Vinz Timmermans,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,Teambuilding,service:4a12cabaa852fa01d8107a68a60f150e,normal,0.0,Internal,False,0.0,0.8,2021-08-25,to_forward,0.0,0.0,34,0.0,8
24104,Vinz Timmermans,Oberon,project:dac6961c7d699864feaad60b7a7437df,Service Team,SERVIC-1,Meetings,service:f0b810be8dd3aaddb48628b99c52fe43,normal,0.0,Internal,False,0.0,0.2,2021-08-25,to_forward,0.0,0.0,34,0.0,8
24105,Vinz Timmermans,Sprout Money BV,project:d546df523896bc3afeaad60b7a7437df,Marktgevoel Agile,MG2021,Marktgevoel - Augustus 2021,service:599b7052835bc71df0d06734914aca76,normal,110.0,Project Management,True,85.0,0.2,2021-08-25,to_forward,0.0,21.2,34,0.0,8


In [49]:
import datetime
one_week_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-1)).strftime(DATE_FORMAT)
five_weeks_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-5)).strftime(DATE_FORMAT)
data = df.query(f'(tariff>0 or service_tariff>0) and day>="{one_week_ago}" and day<"{five_weeks_ago}"')
data

,employee,organization,project_id,project_name,project_number,service,service_id,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,corrections_value,month


In [50]:
df['day'].min()

'2021-01-01'

In [51]:
one_week_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-1)).strftime(DATE_FORMAT)
five_weeks_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-5)).strftime(DATE_FORMAT)
query = f'(tariff>0 or service_tariff>0) and day>="{five_weeks_ago}" and day<"{one_week_ago}"'
print( query )
data = df.query(query)
print( data['day'].min() )
print( data['day'].max() )
print( data['hours'].sum() )
percentage_corrected = 100 * -data['corrections'].sum() / data['hours'].sum()
percentage_corrected

(tariff>0 or service_tariff>0) and day>="2021-07-21" and day<"2021-08-18"
2021-07-21
2021-08-17
2590.75


5.117565698477275

In [52]:
DATE_FORMAT = '%Y-%m-%d'
lastmonth = (datetime.datetime.today() + datetime.timedelta(days=-30)).strftime(DATE_FORMAT)
a = df.query(f'corrections < 0 and day>="{lastmonth}"').groupby(['organization','project_name']).agg({'hours':'sum','corrections':'sum', 'turnover':'sum'}).sort_values('corrections').query('corrections < -10')
a.reset_index()

,organization,project_name,hours,corrections,turnover
0,Sprout Money BV,Marktgevoel Agile,31.5,-31.5,0.0
1,De Volksbank,Eurowijs - SLA,15.2,-13.0,222.8


## Correcties op een specifiek project

In [53]:
easy = df.query('project_number == "EASY-1"')
easy

,employee,organization,project_id,project_name,project_number,service,service_id,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,corrections_value,month
569,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,95.0,Product & UX design,False,85.0,1.0,2021-01-07,projectmanager_approved,0.0,85.0,1,0.0,1
694,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,95.0,UX/UI Design,False,85.0,2.0,2021-01-08,projectmanager_approved,0.0,170.0,1,0.0,1
991,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,95.0,Product & UX design,False,85.0,2.0,2021-01-12,projectmanager_approved,0.0,170.0,2,0.0,1
1038,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,95.0,Product & UX design,False,85.0,3.0,2021-01-12,projectmanager_approved,0.0,255.0,2,0.0,1
1152,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,95.0,Product & UX design,False,85.0,2.0,2021-01-13,projectmanager_approved,0.0,170.0,2,0.0,1
1184,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,95.0,Product & UX design,False,85.0,3.0,2021-01-13,projectmanager_approved,0.0,255.0,2,0.0,1
1258,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,95.0,Product & UX design,False,85.0,2.5,2021-01-14,projectmanager_approved,0.0,212.5,2,0.0,1
1327,Sebastian Schipper,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,110.0,Project Management,False,85.0,2.5,2021-01-14,projectmanager_approved,0.0,212.5,2,0.0,1
1340,Joost Cornelissen,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,125.0,Creative Direction,False,85.0,2.0,2021-01-14,projectmanager_approved,0.0,170.0,2,0.0,1
1745,Patricia Snel,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,95.0,UX/UI Design,False,85.0,4.0,2021-01-19,projectmanager_approved,0.0,340.0,3,0.0,1


In [54]:
easy.groupby(['organization','project_name','project_id']).agg({'hours':'sum','corrections':'sum', 'turnover':'sum'}).query('corrections < -10')

,,,hours,corrections,turnover
organization,project_name,project_id,,,
EasyBroker,Redesign website,project:9b63e1bed41e11eefeaad60b7a7437df,150.5,-11.0,"11,942.5"


In [55]:
df.query(f'corrections < 0 and project_number == "EASY-1"')\
        .groupby(['organization', 'project_name', 'project_id'])\
        .agg({'hours': 'sum', 'corrections': 'sum'})\
        .sort_values('corrections')\
        .reset_index()

,organization,project_name,project_id,hours,corrections
0,EasyBroker,Redesign website,project:9b63e1bed41e11eefeaad60b7a7437df,11.0,-11.0


In [56]:
df.query('corrections < 0').query('project_number == "EASY-1"').groupby(['organization','project_name','project_id']).agg({'hours':'sum','corrections':'sum', 'turnover':'sum'}).query('corrections < -10')

,,,hours,corrections,turnover
organization,project_name,project_id,,,
EasyBroker,Redesign website,project:9b63e1bed41e11eefeaad60b7a7437df,11.0,-11.0,0.0


In [57]:
df.query('corrections < 0 and project_number == "EASY-1"')

,employee,organization,project_id,project_name,project_number,service,service_id,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,corrections_value,month
1746,Sho Stegmeijer,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,95.0,UX/UI Design,False,85.0,3.0,2021-01-19,projectmanager_approved,-3.0,0.0,3,-255.0,1
2545,Sho Stegmeijer,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,95.0,Product & UX design,False,85.0,3.0,2021-01-26,projectmanager_approved,-3.0,0.0,4,-255.0,1
2584,Sho Stegmeijer,EasyBroker,project:9b63e1bed41e11eefeaad60b7a7437df,Redesign website,EASY-1,Redesign website,service:f0b810be8dd3aaddb673c98d2eab4ca5,normal,95.0,Product & UX design,False,85.0,5.0,2021-01-26,projectmanager_approved,-5.0,0.0,4,-425.0,1


## Diensten en omzet per dienst

In [63]:
services_used = set(df['service_id'])
list(services_used)[:3]

['service:e2c6d0787f56872ab48628b99c52fe43',
 'service:9900d01608381b9a12278dc9a55a75e0',
 'service:599b7052835bc71db48628b99c52fe43']

In [85]:
projects = sim.to_pandas(sim.project())[['id','organization_name','name']]
project_names = {row['id']:row['name'] for index, row in projects.iterrows()}
project_organizations = {row['id']:row['organization_name'] for index, row in projects.iterrows()}

service_details = sim.to_pandas(sim.service())[['id','name','project_id','status','revenue_group_label']].query(f"id in {tuple(services_used)}")
service_details

,id,name,project_id,status,revenue_group_label
5,service:86b392ebfd84615bbf0d1878087571b9,Qikker Internal,project:5e920f34c4712abc4c13c77ab857ae53,open,NaN
43,service:10580fb7ced949a2bf0d1878087571b9,Organizational Development,project:5e920f34c4712abc4c13c77ab857ae53,open,NaN
172,service:92e189b9acf2a314d8107a68a60f150e,Strippenkaart 2,project:a019caf7728c511a4c13c77ab857ae53,open,Omzet service
336,service:f8d1fe7ad131e1e2d8107a68a60f150e,Qikker Internal Meetings,project:5e920f34c4712abc4c13c77ab857ae53,open,NaN
393,service:1c9ef73885497a58d8107a68a60f150e,Strippenkaart 2,project:69e5c43660917c504c13c77ab857ae53,open,Omzet service
...,...,...,...,...,...
1044,service:935aeb06941672a3b61e8ec4cf9407af,Development Sprint Design,project:30cdf17b4485b420feaad60b7a7437df,open,Omzet productpropositie
1045,service:935aeb06941672a3783d0f4c8b4f4078,Sprint 110,project:1675e5dd3dbf6d78feaad60b7a7437df,invoiced,Omzet development
1046,service:46b339f1a132a2d0b48628b99c52fe43,Software architectuur,project:b4d3ad8257533e1bfeaad60b7a7437df,invoiced,Omzet productpropositie
1047,service:46b339f1a132a2d0f0d06734914aca76,Development for TLP,project:73e3bddc1e183b9afeaad60b7a7437df,open,Omzet teampropositie


In [89]:
service_details['client'] = service_details.apply(lambda row:project_organizations[row['project_id']], axis=1)
service_details['project'] = service_details.apply(lambda row:project_names[row['project_id']], axis=1)
service_details

,id,name,project_id,status,revenue_group_label,project,client
5,service:86b392ebfd84615bbf0d1878087571b9,Qikker Internal,project:5e920f34c4712abc4c13c77ab857ae53,open,NaN,Internal,Qikker Online B.V.
43,service:10580fb7ced949a2bf0d1878087571b9,Organizational Development,project:5e920f34c4712abc4c13c77ab857ae53,open,NaN,Internal,Qikker Online B.V.
172,service:92e189b9acf2a314d8107a68a60f150e,Strippenkaart 2,project:a019caf7728c511a4c13c77ab857ae53,open,Omzet service,Fietsenmaker-amsterdam.nl Strippenkaart,Alex de Rollende Fietsenmaker
336,service:f8d1fe7ad131e1e2d8107a68a60f150e,Qikker Internal Meetings,project:5e920f34c4712abc4c13c77ab857ae53,open,NaN,Internal,Qikker Online B.V.
393,service:1c9ef73885497a58d8107a68a60f150e,Strippenkaart 2,project:69e5c43660917c504c13c77ab857ae53,open,Omzet service,Iris Nijenhuis Strippenkaart,Iris Nijenhuis
...,...,...,...,...,...,...,...
1044,service:935aeb06941672a3b61e8ec4cf9407af,Development Sprint Design,project:30cdf17b4485b420feaad60b7a7437df,open,Omzet productpropositie,Platform overname,Verhuisdieren.nl
1045,service:935aeb06941672a3783d0f4c8b4f4078,Sprint 110,project:1675e5dd3dbf6d78feaad60b7a7437df,invoiced,Omzet development,Slim Beleggen Agile,Sprout Money BV
1046,service:46b339f1a132a2d0b48628b99c52fe43,Software architectuur,project:b4d3ad8257533e1bfeaad60b7a7437df,invoiced,Omzet productpropositie,Design fases,EasyBroker
1047,service:46b339f1a132a2d0f0d06734914aca76,Development for TLP,project:73e3bddc1e183b9afeaad60b7a7437df,open,Omzet teampropositie,TLP,TUI


In [102]:
revenue_groups = {row['id']:row['revenue_group_label'] for index, row in service_details.iterrows()}

def calc_revenue_group(row):

    if row['project_name'] in ('Slim Beleggen Agile','Slim Beleggen Agile 2020'):
        return 'Omzet trajecten'
    if row['project_name'].count('Travelbase') or row['project_name'].count('TOR-3'):
        return 'Omzet Travelbase'
    rg = service_details_dict[row['service_id']]
    if rg in ('Omzet development', 'Omzet app development','Omzet website','Omzet comcept en design'):
        return 'Omzet projecten'
    return rg

df['revenue_group'] = df.apply(calc_revenue_group, axis=1)

In [103]:

df.groupby(['revenue_group'])[['turnover']].sum()

,turnover
revenue_group,
Omzet Travelbase,"23,210.6"
Omzet concept en design,"18,401.2"
Omzet overig,0.0
Omzet projecten,"1,003,512.9"
Omzet service,"304,471.8"
Omzet trajecten,"528,093.3"


In [104]:
service_details[service_details['revenue_group_label']=='Omzet service']

,id,name,project_id,status,revenue_group_label,project,client
172,service:92e189b9acf2a314d8107a68a60f150e,Strippenkaart 2,project:a019caf7728c511a4c13c77ab857ae53,open,Omzet service,Fietsenmaker-amsterdam.nl Strippenkaart,Alex de Rollende Fietsenmaker
393,service:1c9ef73885497a58d8107a68a60f150e,Strippenkaart 2,project:69e5c43660917c504c13c77ab857ae53,open,Omzet service,Iris Nijenhuis Strippenkaart,Iris Nijenhuis
414,service:6396eb27cf03043cd8107a68a60f150e,Strippenkaart 2,project:59d2ab132db9f9474c13c77ab857ae53,invoiced,Omzet service,Worrell & Jetten Strippenkaart,Worrell & Jetten Accountants en Adviseurs Zoet...
476,service:6ee5e6c75959593fd8107a68a60f150e,Strippenkaart 3,project:fc5dc3ffdea030554c13c77ab857ae53,invoiced,Omzet service,JobOn Strippenkaart,Stichting JobOn
524,service:55647084f3050b86d8107a68a60f150e,Strippenkaart 1 2020,project:20915e733ad449a6feaad60b7a7437df,invoiced,Omzet service,ALDA General Strippenkaart,Alda Nederland B.V.
525,service:798863a14c392243d8107a68a60f150e,Strippenkaart 1,project:a5936e3c2b5e8a37feaad60b7a7437df,invoiced,Omzet service,Maintenance & Support 2020,Greenchoice
541,service:b52d5a2351f25fe2d8107a68a60f150e,Strippenkaart 2,project:13174969648b84f04c13c77ab857ae53,invoiced,Omzet service,MLA Strippenkaart Montessori Lyceum,Montessori Lyceum Amsterdam
598,service:b6b83699fbe3da58d8107a68a60f150e,Strippenkaart 6,project:fa776e37c728faf34c13c77ab857ae53,invoiced,Omzet service,Qpido Strippenkaart,Levvel
646,service:578464fb5901cd91d8107a68a60f150e,Maintenance Sprints,project:f5f44ee31422fb60feaad60b7a7437df,open,Omzet service,Growth Sprints,Chickslovefood B.V.
669,service:474ab70c5156a75dd8107a68a60f150e,Strippenkaart 6,project:823d5a601f8176d44c13c77ab857ae53,invoiced,Omzet service,Kunsten92 Strippenkaart,Kunsten92


# Volgende stap: omzet uit diensten waar geen uren voor zijn gemaakt zoals hosting en onderhuur ook toevoegen. Via Facturen?